# Phase 4: Face Detection & Recognition Evaluation
Using RetinaFace for detection, ArcFace for feature extraction, and FAISS for matching.

**Metrics**: Recognition Accuracy, FAR, FRR, Latency
**Datasets**: LFW, WiderFace (preprocessed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
PROJECT_DIR = '/content/drive/MyDrive/computer_vision'
RESULTS_DIR = f'{PROJECT_DIR}/results/phase4'
os.makedirs(RESULTS_DIR, exist_ok=True)

# Clone repo and download datasets to LOCAL disk (fast SSD)
%cd /content
!rm -rf computer_vision_expirement
!git clone https://github.com/Ib-Programmer/computer_vision_expirement.git
%cd computer_vision_expirement

!pip install -q insightface onnxruntime-gpu faiss-gpu albumentations
!pip install -q -r requirements.txt

# Download and preprocess datasets locally
print("\n--- Downloading datasets to local disk ---")
!python scripts/download_datasets.py lfw widerface
print("\n--- Preprocessing ---")
!python scripts/preprocess_data.py lfw widerface

DATASETS_DIR = '/content/computer_vision_expirement/datasets'
print(f"\nDatasets ready at: {DATASETS_DIR}")
print(f"Results will be saved to Drive: {RESULTS_DIR}")

## 4.1 Face Detection with RetinaFace

In [ ]:
import cv2
import numpy as np
from insightface.app import FaceAnalysis

# Initialize face analysis (RetinaFace + ArcFace)
app = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))
print("InsightFace loaded: RetinaFace (detection) + ArcFace (recognition)")

In [ ]:
import glob
import time

lfw_test = sorted(glob.glob(f'{DATASETS_DIR}/lfw_processed/test/*.jpg'))
print(f"LFW test images: {len(lfw_test)}")

detection_results = []
detection_times = []

for img_path in lfw_test[:200]:  # Process 200 images
    img = cv2.imread(img_path)
    if img is None:
        continue
    
    start = time.time()
    faces = app.get(img)
    elapsed = (time.time() - start) * 1000
    
    detection_times.append(elapsed)
    detection_results.append({
        'path': img_path,
        'num_faces': len(faces),
        'time_ms': elapsed
    })

print(f"Processed: {len(detection_results)} images")
print(f"Avg detection time: {np.mean(detection_times):.1f} ms")
print(f"Faces found: {sum(r['num_faces'] for r in detection_results)}")

## 4.2 Feature Extraction (ArcFace Embeddings)

In [ ]:
# Extract face embeddings from detected faces
embeddings_db = []
labels_db = []

print("Extracting face embeddings from LFW...")
for img_path in lfw_test[:200]:
    img = cv2.imread(img_path)
    if img is None:
        continue
    
    faces = app.get(img)
    for face in faces:
        embedding = face.embedding  # 512-d ArcFace embedding
        label = os.path.basename(os.path.dirname(img_path)) if '/' in img_path else os.path.basename(img_path)
        
        embeddings_db.append(embedding)
        labels_db.append(label)

embeddings_db = np.array(embeddings_db).astype('float32')
print(f"Extracted {len(embeddings_db)} embeddings, shape: {embeddings_db.shape}")

## 4.3 Face Matching with FAISS

In [ ]:
import faiss

if len(embeddings_db) > 0:
    # Normalize embeddings for cosine similarity
    faiss.normalize_L2(embeddings_db)
    
    # Build FAISS index
    dimension = embeddings_db.shape[1]  # 512
    index = faiss.IndexFlatIP(dimension)  # Inner product (cosine after normalization)
    index.add(embeddings_db)
    
    print(f"FAISS index built: {index.ntotal} vectors, {dimension}-d")
    
    # Search: query each embedding against the database
    k = 5  # top-5 matches
    search_start = time.time()
    distances, indices = index.search(embeddings_db[:50], k)  # Query first 50
    search_time = (time.time() - search_start) * 1000
    
    print(f"Search time for 50 queries: {search_time:.1f} ms")
    print(f"Avg per query: {search_time/50:.2f} ms")
    
    # Show sample results
    print("\nSample matches (query -> top match, similarity):")
    for i in range(min(5, len(distances))):
        print(f"  Query {i}: match_idx={indices[i][1]}, similarity={distances[i][1]:.4f}")
else:
    print("No embeddings extracted. Check face detection results.")

## 4.4 Evaluate Recognition Performance

In [ ]:
import pandas as pd

def evaluate_recognition(embeddings, labels, thresholds=np.arange(0.1, 1.0, 0.05)):
    """Evaluate face recognition at different thresholds."""
    faiss.normalize_L2(embeddings.copy())
    
    results = []
    n = len(embeddings)
    
    for threshold in thresholds:
        tp, fp, tn, fn = 0, 0, 0, 0
        
        for i in range(min(n, 100)):  # Sample pairs
            for j in range(i+1, min(n, 100)):
                sim = np.dot(embeddings[i], embeddings[j])
                same_person = (labels[i] == labels[j])
                
                if sim >= threshold:
                    if same_person:
                        tp += 1
                    else:
                        fp += 1
                else:
                    if same_person:
                        fn += 1
                    else:
                        tn += 1
        
        total_genuine = tp + fn
        total_impostor = fp + tn
        
        far = fp / max(total_impostor, 1)
        frr = fn / max(total_genuine, 1)
        accuracy = (tp + tn) / max(tp + fp + tn + fn, 1)
        
        results.append({
            'Threshold': round(threshold, 2),
            'Accuracy': round(accuracy, 4),
            'FAR': round(far, 4),
            'FRR': round(frr, 4),
        })
    
    return pd.DataFrame(results)

if len(embeddings_db) > 0:
    eval_df = evaluate_recognition(embeddings_db, labels_db)
    print("Recognition Performance at Different Thresholds:")
    print(eval_df.to_string(index=False))
    eval_df.to_csv(f'{RESULTS_DIR}/recognition_metrics.csv', index=False)

## 4.5 Test Robustness Under Outdoor Conditions

In [ ]:
import matplotlib.pyplot as plt
import albumentations as A

# Apply outdoor augmentations ON-THE-FLY for robustness testing
conditions = {
    'fog': A.Compose([A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.7, alpha_coef=0.08, p=1.0)]),
    'low_light': A.Compose([A.RandomBrightnessContrast(brightness_limit=(-0.5, -0.2), contrast_limit=(-0.3, 0.0), p=1.0)]),
    'motion_blur': A.Compose([A.MotionBlur(blur_limit=(7, 15), p=1.0)]),
    'rain': A.Compose([A.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1,
                                     drop_color=(200, 200, 200), blur_value=3, brightness_coefficient=0.7, p=1.0)]),
}

# Use LFW test images from local disk
test_imgs = sorted(glob.glob(f'{DATASETS_DIR}/lfw_processed/test/*.jpg'))[:50]
condition_results = []

for condition, transform in conditions.items():
    detected = 0
    total = 0

    for img_path in test_imgs:
        img = cv2.imread(img_path)
        if img is None:
            continue

        # Apply augmentation on-the-fly
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        aug_img = transform(image=img_rgb)['image']
        aug_bgr = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)

        total += 1
        faces = app.get(aug_bgr)
        if len(faces) > 0:
            detected += 1

    rate = detected / max(total, 1) * 100
    condition_results.append({'Condition': condition, 'Images': total, 'Detected': detected, 'Rate': f'{rate:.1f}%'})
    print(f"  {condition}: {detected}/{total} faces detected ({rate:.1f}%)")

if condition_results:
    cond_df = pd.DataFrame(condition_results)
    print("\nRobustness Summary:")
    print(cond_df.to_string(index=False))
    cond_df.to_csv(f'{RESULTS_DIR}/robustness_analysis.csv', index=False)

## 4.6 Literature Comparison (Published Benchmarks)
Comparing our face detection and recognition results with published benchmarks.

In [ ]:
import pandas as pd

# Face Recognition benchmarks on LFW
recognition_lit = pd.DataFrame([
    {'Method': 'DeepFace (Taigman et al., 2014)', 'LFW_Acc%': 97.35, 'Embedding': 4096, 'Loss': 'Softmax', 'Source': 'CVPR 2014'},
    {'Method': 'FaceNet (Schroff et al., 2015)', 'LFW_Acc%': 99.63, 'Embedding': 128, 'Loss': 'Triplet', 'Source': 'CVPR 2015'},
    {'Method': 'SphereFace (Liu et al., 2017)', 'LFW_Acc%': 99.42, 'Embedding': 512, 'Loss': 'A-Softmax', 'Source': 'CVPR 2017'},
    {'Method': 'CosFace (Wang et al., 2018)', 'LFW_Acc%': 99.73, 'Embedding': 512, 'Loss': 'LMCL', 'Source': 'CVPR 2018'},
    {'Method': 'ArcFace (Deng et al., 2019)', 'LFW_Acc%': 99.83, 'Embedding': 512, 'Loss': 'Additive Angular Margin', 'Source': 'CVPR 2019'},
    {'Method': 'AdaFace (Kim et al., 2022)', 'LFW_Acc%': 99.82, 'Embedding': 512, 'Loss': 'Adaptive Margin', 'Source': 'CVPR 2022'},
    {'Method': 'Our ArcFace (InsightFace)', 'LFW_Acc%': None, 'Embedding': 512, 'Loss': 'ArcFace', 'Source': 'This work'},
])

# Face Detection benchmarks on WiderFace
detection_lit = pd.DataFrame([
    {'Method': 'MTCNN (Zhang et al., 2016)', 'Easy': 84.8, 'Medium': 82.5, 'Hard': 60.7, 'Source': 'IEEE SPL 2016'},
    {'Method': 'S3FD (Zhang et al., 2017)', 'Easy': 93.7, 'Medium': 92.4, 'Hard': 85.2, 'Source': 'ICCV 2017'},
    {'Method': 'DSFD (Li et al., 2019)', 'Easy': 96.6, 'Medium': 95.7, 'Hard': 90.4, 'Source': 'CVPR 2019'},
    {'Method': 'RetinaFace (Deng et al., 2020)', 'Easy': 96.9, 'Medium': 96.1, 'Hard': 91.4, 'Source': 'CVPR 2020'},
    {'Method': 'TinaFace (Zhu et al., 2020)', 'Easy': 96.3, 'Medium': 95.7, 'Hard': 92.2, 'Source': 'arXiv 2020'},
    {'Method': 'SCRFD (Guo et al., 2022)', 'Easy': 96.1, 'Medium': 94.9, 'Hard': 88.5, 'Source': 'arXiv 2022'},
    {'Method': 'Our RetinaFace (InsightFace)', 'Easy': None, 'Medium': None, 'Hard': None, 'Source': 'This work'},
])

print('=' * 80)
print('TABLE 4.1: Face Recognition — Published Benchmarks (LFW Accuracy %)')
print('=' * 80)
print(recognition_lit.to_string(index=False))

print('\n' + '=' * 80)
print('TABLE 4.2: Face Detection — Published Benchmarks (WiderFace AP %)')
print('=' * 80)
print(detection_lit.to_string(index=False))

# FAISS retrieval benchmarks
faiss_lit = pd.DataFrame([
    {'Index_Type': 'Flat (brute-force)', 'Recall@1': '100%', 'Search_Time': '~1ms/query (1K vectors)', 'Notes': 'Exact, no approximation'},
    {'Index_Type': 'IVF1024', 'Recall@1': '~95%', 'Search_Time': '~0.1ms/query (1M vectors)', 'Notes': 'Approximate, requires training'},
    {'Index_Type': 'HNSW', 'Recall@1': '~98%', 'Search_Time': '~0.05ms/query (1M vectors)', 'Notes': 'Graph-based, high memory'},
])

print('\n' + '=' * 80)
print('TABLE 4.3: FAISS Index Types — Performance Reference')
print('=' * 80)
print(faiss_lit.to_string(index=False))

# Angular margin loss comparison
print('\n' + '=' * 80)
print('TABLE 4.4: Angular Margin Loss Functions — Evolution')
print('=' * 80)
margin_df = pd.DataFrame([
    {'Loss': 'Softmax', 'Formula': 'log(e^s / sum(e^s))', 'LFW%': '~97.0', 'Key_Innovation': 'Baseline'},
    {'Loss': 'SphereFace (A-Softmax)', 'Formula': 'cos(m*theta)', 'LFW%': '99.42', 'Key_Innovation': 'Multiplicative angular margin'},
    {'Loss': 'CosFace (LMCL)', 'Formula': 'cos(theta) - m', 'LFW%': '99.73', 'Key_Innovation': 'Additive cosine margin'},
    {'Loss': 'ArcFace', 'Formula': 'cos(theta + m)', 'LFW%': '99.83', 'Key_Innovation': 'Additive angular margin (geodesic)'},
])
print(margin_df.to_string(index=False))

# Save all tables
recognition_lit.to_csv(f'{RESULTS_DIR}/literature_face_recognition.csv', index=False)
detection_lit.to_csv(f'{RESULTS_DIR}/literature_face_detection.csv', index=False)
print(f'\nLiterature tables saved to: {RESULTS_DIR}')
print('Note: "This work" rows will be filled after running evaluation sections above.')

In [ ]:
print(f"\nPhase 4 results saved to: {RESULTS_DIR}")
print("Sections completed:")
print("  4.1: Face detection with RetinaFace")
print("  4.2: ArcFace embedding extraction (512-d)")
print("  4.3: FAISS similarity search")
print("  4.4: Recognition threshold evaluation (FAR/FRR)")
print("  4.5: Robustness under outdoor conditions")
print("  4.6: Literature comparison with published benchmarks")
print("Next: Open Phase5_Model_Optimization.ipynb")